In [ ]:
%matplotlib qt5
from process_files import *
from func_science import *

In [ ]:
#
config = configobj.ConfigObj('config.ini')
line = 'Fe_I_6173'
dkdir = config['darks']['directory']
scdir = config['science']['directory']
settings = [f for f in os.listdir(scdir) if 'settings' in f]
settings = scdir + os.sep + settings[0]
settings = configobj.ConfigObj(settings)
# Other numbers
iline = get_line_num(settings, line, 0)
linestr = 'Line_' + str(iline)
nacc = int(settings[linestr]['Polarimeter\\NAccumulations'])
nwav = int(settings[linestr]['NWavePoints'])
filtstr = settings[linestr]['Filter']
modstr = settings[linestr]['Polarimeter\\Modulation']
nmod = int(settings[filtstr]['Polarimeter\\'+modstr+'\\NModulations'])
nfpc = nmod*nacc*nwav
nfpw = nmod*nacc
ncyc = len(os.listdir(scdir+os.sep+line))//3
wavelength = np.float128(settings[linestr]['LineWavelength'])
wave_range = np.float128(settings[linestr]['WaveScanRange'])
wave_step = wave_range/(np.int16(settings[linestr]['NWavePoints'])-1)
geff = np.float128(config['science'][line+'/geff'])
#
# l0corrdir = config['science'][line+'/l0corrdir']
# pol_files = [l0corrdir+os.sep+f for f in sorted(os.listdir(l0corrdir))]
#
l0corrdir_ = config['science'][line+'/l0corrdir_']
pol_files_ = [l0corrdir_+os.sep+f for f in sorted(os.listdir(l0corrdir_))]
#
invdir = config['science']['invdir']
tempdir = os.sep.join(l0corrdir_.split(os.sep)[0:-1])
tempdir_in = tempdir.replace(os.sep+'L0'+os.sep, os.sep+'L1'+os.sep)+os.sep+'tempdir_in' 
tempdir_out = tempdir.replace(os.sep+'L0'+os.sep, os.sep+'L1'+os.sep)+os.sep+'tempdir_out' 
if not os.path.exists(tempdir_in): os.makedirs(tempdir_in)
if not os.path.exists(tempdir_out): os.makedirs(tempdir_out)
#
ff_file = config['flats'][line+'/pol']
tg_file = config['targetplate'][line+'/l0data']

In [ ]:
# Data without image reconstruction
for pfile_ in pol_files_:
    sprefix = ['ii', 'qq', 'uu', 'vv']
    tfile = open(tempdir_in+os.sep+'file_list.txt', 'w')
    sc = pf.open(pfile_)[1].data
    ny, nx, ns, nw = sc.shape
    for s in np.arange(ns):
        for w in np.arange(nw):
            file_name = sprefix[s] + '_' + f'{w:02}' + '.fits'  
            hdu = pf.PrimaryHDU(sc[:,:,s,w])
            hdu.writeto(tempdir_in+os.sep+file_name, overwrite=True)
            tfile.write(file_name + '\n')
    tfile.close()

    # create the map file
    pfile_name = os.path.split(pfile_)[-1]
    params_file = '_'+pfile_name.replace('.fits', '_params.dat')
    stokes_file = '_'+pfile_name.replace('.fits', '_stokes.dat')
    #
    tfile = open(invdir+os.sep+'hellride.map', 'w')
    tfile.write(f'xmin {1} \t \n')
    tfile.write(f'xmax {nx} \t \n')
    tfile.write(f'ymin {1} \t \n')
    tfile.write(f'ymax {ny} \t \n')
    tfile.write(f'dirin {tempdir_in+os.sep} \n')
    tfile.write(f'dirout {tempdir_out+os.sep} \n')
    tfile.write(f'invfileout {params_file} \n')
    tfile.write(f'stokfileout {stokes_file} \n')
    tfile.write(f'fitslist file_list.txt \n')
    tfile.close()

    # create the line file
    noise = 1.0/np.sqrt(sc.mean())
    wave_steps = np.arange(-nwav//2+1, nwav//2+1)*wave_step*1e3
    #
    tfile = open(invdir+os.sep+'hellride.line', 'w')
    tfile.write(f'cen_wav {wavelength:.3f} \n')
    tfile.write(f'geff {geff:.3f} \n')
    tfile.write(f'noise {noise:.5f} \n')
    tfile.write(f'nwav {nwav} \n')
    tfile.write(f'cont_pix {nwav} \n')
    tfile.write(f'wavpos ')
    for w in wave_steps: tfile.write(f'{w} ')
    tfile.write('\n')
    tfile.write('optional hellride \n')
    tfile.close()

    # create the filter file
    et1_r, et2_r = 0.93, 0.93
    et1_s = int(int(settings['Etalon_1']['ZSpacing'])*1e4)
    et2_s = int(int(settings['Etalon_2']['ZSpacing'])*1e4)
    tfile = open(invdir+os.sep+'hellride.filter', 'w')
    tfile.write(f'reflectivity et1 {et1_r:0.3} \n')
    tfile.write(f'reflectivity et2 {et2_r:0.3} \n')
    tfile.write(f'distance plates et1 {et1_s} \n')
    tfile.write(f'distance plates et2 {et2_s} \n')
    tfile.close()
    sp.run(f'mpirun -n 16 {invdir}/vfisv_fpi.x -map={invdir}/hellride.map -line={invdir}/hellride.line -filter={invdir}/hellride.filter', shell=True)

In [ ]:
# Image reconstructed data
for pfile in pol_files[30:40]:
    sprefix = ['ii', 'qq', 'uu', 'vv']
    tfile = open(tempdir_in+os.sep+'file_list.txt', 'w')
    sc = pf.open(pfile)[1].data
    ny, nx, ns, nw = sc.shape
    for s in np.arange(ns):
        for w in np.arange(nw):
            file_name = sprefix[s] + '_' + f'{w:02}' + '.fits'  
            hdu = pf.PrimaryHDU(sc[:,:,s,w])
            hdu.writeto(tempdir_in+os.sep+file_name, overwrite=True)
            tfile.write(file_name + '\n')
    tfile.close()

    # create the map file
    pfile_name = os.path.split(pfile)[-1]
    params_file = pfile_name.replace('.fits', '_params.dat')
    stokes_file = pfile_name.replace('.fits', '_stokes.dat')
    #
    tfile = open(invdir+os.sep+'hellride.map', 'w')
    tfile.write(f'xmin {1} \t \n')
    tfile.write(f'xmax {nx} \t \n')
    tfile.write(f'ymin {1} \t \n')
    tfile.write(f'ymax {ny} \t \n')
    tfile.write(f'dirin {tempdir_in+os.sep} \n')
    tfile.write(f'dirout {tempdir_out+os.sep} \n')
    tfile.write(f'invfileout {params_file} \n')
    tfile.write(f'stokfileout {stokes_file} \n')
    tfile.write(f'fitslist file_list.txt \n')
    tfile.close()

    # create the line file
    noise = 0.001
    wave_steps = np.arange(-nwav//2+1, nwav//2+1)*wave_step*1e3
    #
    tfile = open(invdir+os.sep+'hellride.line', 'w')
    tfile.write(f'cen_wav {wavelength:.3f} \n')
    tfile.write(f'geff {geff:.3f} \n')
    tfile.write(f'noise {noise:.5f} \n')
    tfile.write(f'nwav {nwav} \n')
    tfile.write(f'cont_pix {nwav} \n')
    tfile.write(f'wavpos ')
    for w in wave_steps: tfile.write(f'{w} ')
    tfile.write('\n')
    tfile.write('optional hellride \n')
    tfile.close()

    # create the filter file
    et1_r, et2_r = 0.93, 0.93
    et1_s = int(int(settings['Etalon_1']['ZSpacing'])*1e4)
    et2_s = int(int(settings['Etalon_2']['ZSpacing'])*1e4)
    tfile = open(invdir+os.sep+'hellride.filter', 'w')
    tfile.write(f'reflectivity et1 {et1_r:0.3} \n')
    tfile.write(f'reflectivity et2 {et2_r:0.3} \n')
    tfile.write(f'distance plates et1 {et1_s} \n')
    tfile.write(f'distance plates et2 {et2_s} \n')
    tfile.close()
    sp.run(f'mpirun -n 16 {invdir}/vfisv_fpi.x -map={invdir}/hellride.map -line={invdir}/hellride.line -filter={invdir}/hellride.filter', shell=True)

In [ ]:
sc = pf.open(pol_files[30])[0].data
plt.imshow(sc)
# show_img_series(sc[:,:,0])

In [ ]:
# For template spectrum from the 
lines_ff = pf.open(ff_file)
align_pars = pf.open(tg_file)
affine12 = align_pars[3].data[1]
#
lines_pol1 = lines_ff[7].data
lines_pol2 = lines_ff[8].data
for w in range(nwav):
    lines_pol2[:,:,w] = affine_transform(lines_pol2[:,:,w], affine12, mode='nearest')
lines_pol = 0.5*(lines_pol1+lines_pol2)
zeros = 0*lines_pol
lines_pol_ = np.array([lines_pol, zeros, zeros, zeros])
lines_pol_ = np.moveaxis(lines_pol_, 0, 2)

In [ ]:
# For template spectrum from the 
lines_ff = pf.open(ff_file)
lines_pol1 = lines_ff[7].data
zeros = 0*lines_pol1
lines_pol1 = np.array([lines_pol1, zeros, zeros, zeros])
lines_pol1 = np.moveaxis(lines_pol1, 0, 2)
lines_pol2 = lines_ff[8].data
lines_pol2 = np.array([lines_pol2, zeros, zeros, zeros])
lines_pol2 = np.moveaxis(lines_pol2, 0, 2)
#
for ff, prefix in zip([lines_pol1, lines_pol2], ['pol1', 'pol2']):
    sprefix = ['ii', 'qq', 'uu', 'vv']
    tfile = open(tempdir_in+os.sep+'file_list.txt', 'w')
    # sc = pf.open(pfile)[1].data
    ny, nx, ns, nw = ff.shape
    for s in np.arange(ns):
        for w in np.arange(nw):
            file_name = sprefix[s] + '_' + f'{w:02}' + '.fits'  
            hdu = pf.PrimaryHDU(ff[:,:,s,w])
            hdu.writeto(tempdir_in+os.sep+file_name, overwrite=True)
            tfile.write(file_name + '\n')
    tfile.close()

    # create the map file
    params_file = prefix + '_flat_params.fits'
    stokes_file = prefix + '_flat_stokes.fits'
    #
    tfile = open(invdir+os.sep+'hellride.map', 'w')
    tfile.write(f'xmin {1} \t \n')
    tfile.write(f'xmax {nx} \t \n')
    tfile.write(f'ymin {1} \t \n')
    tfile.write(f'ymax {ny} \t \n')
    tfile.write(f'dirin {tempdir_in+os.sep} \n')
    tfile.write(f'dirout {tempdir_out+os.sep} \n')
    tfile.write(f'invfileout {params_file} \n')
    tfile.write(f'stokfileout {stokes_file} \n')
    tfile.write(f'fitslist file_list.txt \n')
    tfile.close()

    # create the line file
    noise = 0.001
    wave_steps = np.arange(-nwav//2+1, nwav//2+1)*wave_step*1e3
    #
    tfile = open(invdir+os.sep+'hellride.line', 'w')
    tfile.write(f'cen_wav {wavelength:.3f} \n')
    tfile.write(f'geff {geff:.3f} \n')
    tfile.write(f'noise {noise:.5f} \n')
    tfile.write(f'nwav {nwav} \n')
    tfile.write(f'cont_pix {nwav} \n')
    tfile.write(f'wavpos ')
    for w in wave_steps: tfile.write(f'{w} ')
    tfile.write('\n')
    tfile.write('optional hellride \n')
    tfile.close()

    # create the filter file
    et1_r, et2_r = 0.93, 0.93
    et1_s = int(int(settings['Etalon_1']['ZSpacing'])*1e4)
    et2_s = int(int(settings['Etalon_2']['ZSpacing'])*1e4)
    tfile = open(invdir+os.sep+'hellride.filter', 'w')
    tfile.write(f'reflectivity et1 {et1_r:0.3} \n')
    tfile.write(f'reflectivity et2 {et2_r:0.3} \n')
    tfile.write(f'distance plates et1 {et1_s} \n')
    tfile.write(f'distance plates et2 {et2_s} \n')
    tfile.close()
    sp.run(f'mpirun -n 16 {invdir}/vfisv_fpi.x -map={invdir}/hellride.map -line={invdir}/hellride.line -filter={invdir}/hellride.filter', shell=True)

In [ ]:
lines_pol1.shape